Nystro🐈Nyan Nyora [WIP?]


The work is based on
* idea of LoRA (take little parameters to represent big matrix)
* idea of Nyströmformer and Nyström approximation (take 1 matrix, beat it into smaller parts)

SPOILER: Pretty shitty result once I used real model, oh well, worth trying.

All hope abandon ye who enter here further, I'ma keeping it in case which likely not to happen or to base other impls

(PLACEHOLDER FOR A KITTEN OR A CATGIRL)

In [6]:
# imports,  let's skip this part, it's not my favorite
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling, BitsAndBytesConfig

In [1]:
# Settings
from pathlib import Path  #(only place where it will be used )

MODEL_PATH = Path("~/models/TinyLlama_TinyLlama-1.1B-intermediate-step-1431k-3T").expanduser()
DTYPE = torch.bfloat16
ATTN_IMPL="flash_attention_2"
DEVICE = "cuda"
RANK=32
BATCH_SIZE=3
N_CTX=640
DATASET = "boolq"

SPOILER: In the end I've found that LiteLlama tokenizer eats double quote '"'...

(https://huggingface.co/ahxt/LiteLlama-460M-1T/discussions/9#659f6f894c074ce5e4e9532c)

... as it thinks it's a [PAD], so results are questionable, need to check on other model

In [168]:
# Let's build Nystronyan block that uses less than (NxN) parameters to be 
# To read inspration, check https://arxiv.org/pdf/2102.03902.pdf, however we need only a single formula from there
# as we don't care that much of finer details such as "optimization"
class NystroNyan(nn.Module):
    def __init__(self, in_features: int, out_features: int, r: int):
        
        super().__init__()
        # mirror nn.Linear reports
        self.in_features = in_features
        self.out_features = out_features
        self.rank = r

        # big matrix n x n, split into four parts
        #   
        #  (r r)          (r, n_col-r) 
        #    A                  B
        #    F                  C
        #  (n_row-r r)  (n_row-r n_col-r)
        # our part of interest  is C.

        self.A = nn.Parameter(torch.rand(r, r))
        self.B = nn.Parameter(torch.rand(r, out_features-r))
        self.F = nn.Parameter(torch.rand(in_features-r, r))
        
    @property
    def A_(self):
        # pinverse doesn't support bf16
        return self.A.float().pinverse().to(dtype=self.A.dtype)

    def forward(self, x):
        # we need Moore-Penrose inverse
        A_ = self.A_

        AB = torch.cat((self.A, self.B), -1)  # AB upper half
        AF = torch.cat((self.A, self.F), 0)   # AF left half
        
        reconsturcted_matrix = AF @ A_ @ AB

        return x @ reconsturcted_matrix



In [4]:
# Sanity check
def model_numel(m, grad_only=False):
    return sum(p.numel() for p in m.parameters() if p.requires_grad or not grad_only)
    
def sanity_check(n=4096, r=32):
    print(f"\n({n=}, {r=})")
    print("NystroNyan:\t", model_numel(NystroNyan(n, n, r)))
    print("LoRA:\t", torch.zeros(2, n, r).numel())

sanity_check(r=8)
sanity_check(r=16)
sanity_check(r=32)
# We are more parameter efficient. Nyaa.


(n=4096, r=8)
NystroNyan:	 65472
LoRA:	 65536

(n=4096, r=16)
NystroNyan:	 130816
LoRA:	 131072

(n=4096, r=32)
NystroNyan:	 261120
LoRA:	 262144


In [2]:
# Loading a model
def reset_model():
    global model

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=DTYPE
    )   
     
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,        
        torch_dtype=DTYPE,
        attn_implementation=ATTN_IMPL,
        quantization_config=bnb_config,
        trust_remote_code=True
    )
    #model.to(device=DEVICE)
    return model
reset_model()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear4bit(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaR

In [3]:
# Loading a tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
if tokenizer.pad_token_id is None:    
    tokenizer.pad_token_id = tokenizer.unk_token_id
tokenizer_path = MODEL_PATH

In [127]:
# For data loading
def tokenize_text(samples, tokenizer):
    return {"input_ids": tokenizer(samples["text"]).input_ids}

def tokenize_boolq(samples, tokenizer):
    n_samples = len(samples["question"])
    prompt = "\n\n\n### Passage:\n{0}\n\n### Question:\n{1}\n\n### Answer:\n{2}"
    preprompt = "".join([
        prompt.format("Stockholm is the capital of Sweden", "Is Stockholm the capital of Sweden", "True"),
        prompt.format("3/5 is greater than -5/3", "Is -5/3 greater than 3/5", "False")
    ])
    data = []
    for i in range(n_samples):
        passage, question, answer = samples["passage"][i], samples["question"][i], samples["answer"][i]
        passage = passage[:1000]
        new_sample = preprompt + prompt.format(passage, question, answer)
        data.append(new_sample)
    return {"input_ids": tokenizer(data, return_attention_mask=False).input_ids}

In [130]:
# for data loading
def get_dl(batch_size, split="train"):
    ds = datasets.load_dataset(DATASET)
    ds = ds[split]    
    ds = ds.map(
        tokenize_boolq,
        remove_columns=ds.column_names,
        fn_kwargs={"tokenizer": tokenizer},
        batched=True)
    collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
    dl = DataLoader(
        ds,
        batch_size=batch_size,
        collate_fn=collator,
        shuffle=False)
    return dl

import functools

# We cache result to omit stupid messages like "Found cached dataset parquet"
@functools.cache
def get_dl_cached(batch_size, split="train"):
    return get_dl(batch_size, split)

def iter_batch(batch: dict, n_ctx: int):
    for start in range(0, batch.input_ids.shape[1], n_ctx):
        part = {k: v[:, start:start+n_ctx].to(DEVICE) for k,v in batch.items()}
        part["progress"] = (start, batch.input_ids.shape[1])
        yield part

In [171]:
# Now we need a module for injecting our cute kitten
class NyanInjector(nn.Module):
    def __init__(self, base: nn.Linear, r: int) -> None:
        super().__init__()
        assert isinstance(base, nn.Linear)
        self.base = base
        self.delta = NystroNyan(base.in_features, base.out_features, r)
        for p in self.delta.parameters():
            p.data *= model.config.initializer_range
        
    def forward(self, x):
        y = self.base(x)
        y = y + self.delta(x)
        return y



In [172]:
# Freeze everything ❄
model.requires_grad_(False)

# Hot catgirls go in 🔥
for layer in model.model.layers:
    attn = layer.self_attn
    assert hasattr(attn, "q_proj")
    assert hasattr(attn, "v_proj")
    attn.q_proj = NyanInjector(attn.q_proj, r=RANK).to(dtype=DTYPE, device=DEVICE)
    attn.v_proj = NyanInjector(attn.v_proj, r=RANK).to(dtype=DTYPE, device=DEVICE)

parms_to_train = model_numel(model, grad_only=True)
parms_total = model_numel(model, grad_only=False) 
print(f"#parms: {parms_total}, trainable: {parms_to_train} ({parms_to_train/parms_total:%})")



#parms: 620066816, trainable: 4460544 (0.719365%)


In [11]:
def nyan_dict():
    all_dict = model.state_dict()
    to_delete = [k for k in all_dict.keys() if ".delta." not in k]
    for k in to_delete:
        del all_dict[k]
    return all_dict



In [12]:
torch.save(nyan_dict(), "weights/phi.nyan-4bit.bin")

In [173]:
dl = get_dl(batch_size=BATCH_SIZE)

In [178]:
opt = torch.optim.AdamW(model.parameters())

for ib, b in enumerate(bar := tqdm(dl)):
    for mb in iter_batch(b, N_CTX):
        (cur, total) = mb.pop("progress")
        loss = model(**mb).loss
        loss.backward()
        opt.step()
        opt.zero_grad()
        bar.set_description(f'L:{loss.item():.4} P:{cur/total:%}')
    #if ib % 100 == 0:
    #    torch.save(nyan_dict(), f"weights/ollama3.nyan-0.4bit-E{ib}.bin")
#torch.save(nyan_dict(), f"weights/ollama3.nyan-0.4bit-E{ib}.bin")


  0%|          | 0/3143 [00:00<?, ?it/s]

In [ ]:
# unload nyora
for layer in model.model.layers:
    attn = layer.self_attn
    assert isinstance(attn.q_proj, NyanInjector)
    attn.q_proj = attn.q_proj.base
    attn.v_proj = attn.v_proj.base
    

In [177]:
from peft import LoraConfig, get_peft_model, TaskType
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, 
                         inference_mode=False, r=RANK, 
                         lora_alpha=32, 
                         lora_dropout=0.1, 
                         target_modules=['q_proj', 'v_proj'])
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 4505600 || all params: 620111872 || trainable%: 0.7265785745188894


In [181]:
@torch.no_grad()
def run_valid_acc(strict=False):
    assert strict == False, "text sampling NYI"
    t_true = tokenizer.encode("\nTrue", add_special_tokens=False)
    t_true = t_true[-1]
    t_false = tokenizer.encode("\nFalse", add_special_tokens=False)
    t_false = t_false[-1]
    

    model.eval()
    ds = datasets.load_dataset(DATASET, split="validation")
    total = 0
    correct = 0
    correct_trues = 0
    correct_falses = 0
    # todo: generate/w max_new_tokens=1?
    for i in tqdm(range(len(ds))):
        total += 1
        samples = ds[i:i+1]
        inputs = tokenize_boolq(samples, tokenizer)

        y_true = inputs["input_ids"][0].pop()
        assert y_true in [t_true, t_false], f"{y_true} != {t_true}/{t_false}"
        y_true = y_true == t_true
        inputs = torch.tensor(inputs["input_ids"]).to(device=DEVICE)
        out = model(input_ids=inputs).logits[0, -1]
        out = nn.functional.softmax(out, -1)
        if not strict:
            y_pred = out[t_true] > out[t_false]
        else:
            raise NotImplementedError("Sttrict acc mode nyi")
        correct += y_true == y_pred
        if y_true == y_pred:
            if y_true:
                correct_trues += 1
            else:
                correct_falses += 1
        

    #model.train()
    print(f"{correct}/{total}={correct/total:%} CT:{correct_trues} CF:{correct_falses}")
    return correct/total
            

run_valid_acc()

# Vanilla: 
# Raw    : 1389/3270=42.477062%
# Nyaron : 1822/3270=55.718654%
# LoRA   : 2359/3270=72.140670%




  0%|          | 0/3270 [00:00<?, ?it/s]

2359/3270=72.140670% CT:1575 CF:784


tensor(0.7214, device='cuda:0')

In [176]:
reset_model()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear4bit(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaR

In [ ]:
for x in get_dl_cached(split="validation", batch_size=1):
    print(model.config)

    break

In [182]:
def sanity_prompt(p):
    x = tokenizer(p, return_tensors="pt").to(DEVICE)
    #del x["token_type_ids"]
    x = model.generate(**x, max_new_tokens=20, do_sample=False).ravel()
    x2 = tokenizer.decode(x)
    return x,x2
x1, x2 = sanity_prompt("""A tabby cat sits on the wall, looking""")
print(x2)

<s> A tabby cat sits on the wall, looking at the camera.
After a few seconds, the cat jumps down and runs to the other
